# Procedimiento: Creación, Configuración y Lanzamiento de un Paquete ROS 2

Este documento detalla los pasos críticos seguidos para construir el paquete `mi_primer_paquete`, configurar un nodo con parámetros y ejecutarlo mediante un archivo **Launch**.

## 1. Estructura del Workspace
La raíz del workspace es el directorio principal (ej. `~/ros2_workspace`). Para que ROS 2 funcione, la estructura debe ser:
```text
ros2_workspace/
├── src/                  # Código fuente
│   └── mi_primer_paquete/
├── build/                # Archivos temporales de compilación
├── install/              # Ejecutables y recursos instalados
└── log/                  # Registros de errores y eventos
```

--- 

## 2. Configuración Crítica de Archivos

### A. `package.xml` (Dependencias)
Es el DNI del paquete. Sin las dependencias de ejecución, el sistema no sabrá qué librerías cargar.
**Líneas críticas:**
```xml
<exec_depend>rclpy</exec_depend>
<exec_depend>launch_ros</exec_depend>
```
* `rclpy`: Necesaria para nodos Python.
* `launch_ros`: Necesaria para que el comando `ros2 launch` reconozca los nodos.

### B. `setup.py` (Instalación y Entry Points)
Este archivo indica a `colcon` qué instalar y cómo llamar a los scripts.
**Líneas críticas:**
1. **Imports de sistema:**
   ```python
   import os
   from glob import glob
   ```
2. **Instalación del Launch file (en `data_files`):**
   ```python
   (os.path.join('share', package_name, 'launch'), glob(os.path.join('launch', '*launch.[pxy][yma]*'))),
   ```
3. **Registro del ejecutable (en `entry_points`):**
   ```python
   'mi_nodo_param = mi_primer_paquete.mi_nodo_param:main',
   ```

### C. `mi_lanzamiento_launch.py` (Orquestación)
Permite lanzar múltiples nodos con configuraciones específicas.
**Líneas críticas:**
```python
Node(
    package='mi_primer_paquete',      # Debe coincidir con el nombre en setup.py
    executable='mi_nodo_param',       # Debe coincidir con el entry_point
    parameters=[{'mensaje': '...', 'frecuencia': 2.0}] # Configuración dinámica
)
```

--- 

## 3. El Nodo con Parámetros (`mi_nodo_param.py`)
Un nodo profesional debe ser configurable. En el código, esto se logra en dos pasos clave:
1. **Declaración:** `self.declare_parameter('nombre', valor_defecto)`
2. **Obtención:** `self.get_parameter('nombre').get_parameter_value()`

--- 

## 4. Flujo de Trabajo en Terminal (Procedimiento)

Para aplicar cambios y ejecutar, siempre se sigue este orden desde la **raíz del workspace**:

1. **Compilar:**
   ```bash
   colcon build --packages-select mi_primer_paquete
   ```
   *Nota: Si falló antes, usa `rm -rf build/ install/` para limpiar.*

2. **Cargar el Entorno (Overlay):**
   ```bash
   source install/setup.bash
   ```
   *Este comando es el que permite que `ros2 launch` encuentre tu paquete local.*

3. **Ejecutar:**
   ```bash
   ros2 launch mi_primer_paquete mi_lanzamiento_launch.py
   ```

--- 

## 5. Resumen de Errores Comunes
* **NameError: name 'os' is not defined**: Falta `import os` en el `setup.py`.
* **Package not found**: No se ha hecho `source install/setup.bash` o el nombre del paquete en el archivo Launch es incorrecto.
* **Executable not found**: El nombre en el Launch file no coincide con el definido en `entry_points` del `setup.py`.